In [ ]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm.auto import tqdm

def get_top_k_aip_lamp2(query,docs,categories,scores,k):
    res = sorted(list(zip(docs,categories,scores)),key = lambda x:x[2],reverse=True)
    final = ""
    for i in range(min(k,len(scores))):
        ppep = 'the category for the article:"'+res[i][0]+'" is "'+res[i][1]+'", and '
        final +=ppep
    final = final[:-6] + "."+query
    return final

def get_top_k_aip_lamp4(query,docs,title,scores,k):
    res = sorted(list(zip(docs,title,scores)),key = lambda x:x[2],reverse=True)
    final = ""
    for i in range(min(k,len(scores))):
        ppep = '"'+ res[i][1]+'" is the title for "' + res[i][0] + '", and '
        final +=ppep
    final = final[:-6] + "."+query
    return final

def bert(all_data, tokenizer, model):

    f_aip = open(path + aip, "a+")
    #f_score = open("drive/MyDrive/646:IR/scores.txt", "a+")

    progress_bar = tqdm(range(2000))

    txt = "the category for the article: "
    for instance in all_data:
        query = instance['query']
        document_tokens = tokenizer.batch_encode_plus(instance['docs'], padding=True, truncation=True, return_tensors='pt').to('cuda')
        query_tokens = tokenizer.encode_plus(query, padding=True, truncation=True, return_tensors='pt').to('cuda')
        document_embeddings = model(**document_tokens).last_hidden_state.mean(dim=1)  # Average pooling over tokens
        query_embedding = model(**query_tokens).last_hidden_state.mean(dim=1).squeeze()
        similarities = cosine_similarity(query_embedding.reshape(1, -1).detach().cpu().numpy(), document_embeddings.detach().cpu().numpy()).squeeze()
        # cur_aip = get_top_k_aip_lamp2(query, instance['docs'], instance['titles'], similarities, 3)
        cur_aip = get_top_k_aip_lamp4(query, instance['docs'], instance['titles'], similarities, 3)
        f_aip.write(cur_aip+"----")
        progress_bar.update(1)

    f_aip.close()


In [ ]:
import json
import numpy as np
import time
import pandas as pd


def get_llm_input():
    f = open(path + aip,"r")
    aip_text = f.read().split("----")
    aip_text.pop()
    len(aip_text)
    f2 = open(path + output)
    y_true = json.load(f2)
    y_true = y_true["golds"]

    y_true = [i['output'] for i in y_true]
    y_true = y_true
    print(len(aip_text))
    llm_data = pd.DataFrame({
        'text':aip_text,
        'label':y_true
    })
    llm_data.to_csv(path + llm_input)

In [ ]:
prefix_path = 'drive/MyDrive/646:IR/'
dataset = 'Lamp4U/' #change
mode = 'vali/' #change train or vali
data = 'bm25_hnsw_top_5.json'
aip = 'aip.txt'
output = 'dev_outputs.json'
llm_input = 'bm25_hnsw_llm.csv'

path = prefix_path + dataset + mode
file_path = path + 'bm25_hnsw_top_5.json'
bm25_hnsw_res = None
with open(file_path, 'r') as file:
  bm25_hnsw_res = json.load(file)
print(len(bm25_hnsw_res))


1925


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to('cuda')

bert(bm25_hnsw_res,tokenizer,model)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
get_llm_input()

1925
